Ok, now we can create a simple logistic regression model with TensorFlow directly. We already saw logistic regression with Keras on top of TensorFlow, so this is digging in at the base level.

You'll quickly see it's a lot more code.

In [1]:
import tensorflow as tf

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Everyone has MNIST digits!

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
num_inputs = 28 * 28 # MNIST image size
num_outputs = 10 # 10 output digits
batch_size = 64 # mini batch
epochs = 10 # total training loops
learning_rate = 0.01 # amount we update parameters

Inputs, these are placeholders that will take the images, flattened to arrays and the ouput digits encoded one hot.

In [4]:
x = tf.placeholder(tf.float32, [None, num_inputs]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, num_outputs]) # 0-9 digits recognition => 10 classes

Learned variables, going from 784 -- input size -- to the 10 output classes

In [5]:
W = tf.Variable(tf.zeros([num_inputs, num_outputs]))
b = tf.Variable(tf.zeros([num_outputs]))

This is the actual model, a one liner that jsut runs softmax, which is conveniently provided!

In [6]:
pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax

And the basic cookbook, a loss and an optimizer. This is bit of math, as you need to provide your own categorical cross entropy function -- the loss. But the 

In [7]:
# Minimize error using cross entropy
loss = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

Need to initialize those variables!

In [8]:
init = tf.global_variables_initializer()

And now the training loop. Compared to MxNet, you'll notice the backward propagation and computation of the loss is somewhat automatic. You just tell the session to run with the optimizer and the loss function.

The way I think about is is the session 'pulls' data from the feed -- batch -- through the `loss` function, which pulls on the model, here via `pred`, which pulls on `x` and `y`.

In [9]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training cycle
    for epoch in range(epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, loss], feed_dict={x: batch_xs,
                                                          y: batch_ys})

        print("Epoch:", '%04d' % epoch, "cost=", "{:.9f}".format(c))
        # Calculate accuracy
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

    

Epoch: 0000 cost= 0.644322157
Accuracy: 0.8666
Epoch: 0001 cost= 0.388087213
Accuracy: 0.883
Epoch: 0002 cost= 0.463756293
Accuracy: 0.8896
Epoch: 0003 cost= 0.456716388
Accuracy: 0.8944
Epoch: 0004 cost= 0.367341220
Accuracy: 0.8996
Epoch: 0005 cost= 0.460425138
Accuracy: 0.9027
Epoch: 0006 cost= 0.228121877
Accuracy: 0.9039
Epoch: 0007 cost= 0.318341345
Accuracy: 0.9056
Epoch: 0008 cost= 0.475308716
Accuracy: 0.9079
Epoch: 0009 cost= 0.262421548
Accuracy: 0.9087


Good old logisti regression -- but quite a bit more code than Keras!